<a href="https://colab.research.google.com/github/vspvikram/AML_project/blob/main/shreya_model_AML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import os
import numpy as np
from tensorflow.keras.layers import Dropout
import datetime
import time
import re
import functools
import zipfile

import cv2
import tensorflow as tf

from PIL import Image
from bs4 import BeautifulSoup
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from collections import Counter

import string

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.preprocessing.image import load_img
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [38]:
train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/train.csv')  
test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/test.csv') 
train.head()

,Image1,Image2,Caption,Number,Clean_capt,Caption_count
0,CXR588_IM-2183-1001,CXR588_IM-2183-2001,No acute cardiopulmonary abnormalities.,831,no acute cardiopulmonary abnormality,117
1,CXR2937_IM-1339-1001,CXR2937_IM-1339-2001,No acute cardiopulmonary findings.,396,no acute cardiopulmonary finding,118
2,CXR255_IM-1058-1001,CXR255_IM-1058-2001,No acute cardiopulmonary disease.,2281,no acute cardiopulmonary disease,105
3,CXR1679_IM-0448-1001,CXR1679_IM-0448-1002,Normal chest,1779,normal chest,36
4,CXR2660_IM-1142-1001,CXR2660_IM-1142-2001,Normal chest,518,normal chest,36


In [7]:
print(len(train['Number'].unique()))
print(len(train['Number']))

3086
3214


In [4]:
#!unzip "/content/gdrive/MyDrive/Colab Notebooks/image.zip" -d "/content/gdrive/MyDrive/Colab Notebooks/image/"

Archive:  /content/gdrive/MyDrive/Colab Notebooks/image.zip
replace /content/gdrive/MyDrive/Colab Notebooks/image/CXR163_IM-0410-12012.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [39]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image


model = InceptionV3(weights='imagenet')
model_new = Model(model.input, model.layers[-2].output)
for layer in model_new.layers:
    layer.trainable=False

'''
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def encode(image):
    image = preprocess(image) 
    fea_vec = model_new.predict(image) 
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    return fea_vec

encoding_train = {}
for img in train_img:
    encoding_train[img[len(images_path):]] = encode(img)
train_features = encoding_train

encoding_test = {}
for img in test_img:
    encoding_test[img[len(images_path):]] = encode(img)
'''

'\ndef preprocess(image_path):\n    img = image.load_img(image_path, target_size=(224, 224))\n    x = image.img_to_array(img)\n    x = np.expand_dims(x, axis=0)\n    x = preprocess_input(x)\n    return x\n\ndef encode(image):\n    image = preprocess(image) \n    fea_vec = model_new.predict(image) \n    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])\n    return fea_vec\n\nencoding_train = {}\nfor img in train_img:\n    encoding_train[img[len(images_path):]] = encode(img)\ntrain_features = encoding_train\n\nencoding_test = {}\nfor img in test_img:\n    encoding_test[img[len(images_path):]] = encode(img)\n'

In [40]:
def get_img_features(df, input_size):
    path = '/content/gdrive/MyDrive/Colab Notebooks/image/'
    
    Xnet_features = pd.DataFrame(columns=['Number','features'])
    pos = 0
    for i in range(len(df)):
        f = os.path.join(path, df.iloc[i]["Image1"].lstrip())
        f = f+".png"
        #image1 = Image.open(f)
        img1 = image.load_img(f, target_size=(299, 299))
        x1 = image.img_to_array(img1)
        x1 = np.expand_dims(x1, axis=0)
        x1 = preprocess_input(x1)
        
        f = os.path.join(path, df.iloc[i]["Image2"].lstrip())
        f = f+".png"
        
        img2 = image.load_img(f, target_size=(299, 299))
        x2 = image.img_to_array(img2)
        x2 = np.expand_dims(x2, axis=0)
        x2 = preprocess_input(x2)

        #image2 = Image.open(f)
        
        def encode(image):
            fea_vec = model_new.predict(image) 
            #fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
            return fea_vec

        image1_features = encode(x1)
        image2_features = encode(x2)
        input_concat = np.concatenate((image1_features, image2_features), axis=1)
        
        Xnet_features.loc[pos] = [int(df.iloc[i]['Number']),input_concat]
        pos=pos+1
    return Xnet_features

In [23]:
a = {}
a[12]='apple'
a[12]='ball'
a

{12: 'ball'}

In [41]:
en_output = get_img_features(train.iloc[0:40,], (299,299,3))
en_output

,Number,features
0,831,"[[0.37776634, 0.37836018, 0.082196, 0.2683568,..."
1,396,"[[0.5769105, 0.30342862, 0.15529898, 0.3956436..."
2,2281,"[[0.2971441, 0.35165608, 0.04101849, 0.3135675..."
3,1779,"[[0.61468536, 0.39148712, 0.2238153, 0.2804152..."
4,518,"[[0.45647693, 0.32283726, 0.23408943, 0.260987..."
5,3746,"[[0.13502342, 0.4192431, 0.08697965, 0.2309943..."
6,472,"[[0.64468104, 0.7292648, 0.22713414, 0.3975431..."
7,80,"[[0.58057076, 0.33272293, 0.22853439, 0.198079..."
8,316,"[[0.9567922, 0.12568782, 0.073065616, 0.208439..."
9,1810,"[[0.65189403, 0.24754304, 0.12505412, 0.369573..."


In [69]:
en_output['features'][0][0][0]

0.37776634

In [ ]:
def preprocess_text(text_df):
  porter = WordNetLemmatizer()
  punct = str.maketrans('', '', string.punctuation)
  i=0
  for description in text_df:
    pre_processed_words=[]
    words=description.split()
    words=[word.lower() for word in words]
    words =[word.translate(punct) for word in words]
    words=[word for word in words if word.isalpha()]
    words=[porter.lemmatize(word) for word in words]
    words=[word for word in words if len(word)>1]
    words='start '+' '.join(words)+' end'
    pre_processed_words.append(words)
    text_df[i]=pre_processed_words
    i=i+1
  return text_df
text_dict=preprocess_text(train['Caption'])

In [43]:
text_dict

0        [start no acute cardiopulmonary abnormality end]
1            [start no acute cardiopulmonary finding end]
2            [start no acute cardiopulmonary disease end]
3                                [start normal chest end]
4                                [start normal chest end]
                              ...                        
3209    [start borderline cardiomegaly no acute findin...
3210    [start heart size normal tortuous dilated aort...
3211         [start xxxx xxxx right pleural effusion end]
3212    [start decrease illdefined mixed lucent and op...
3213    [start moderate sized right pneumothorax there...
Name: Caption, Length: 3214, dtype: object

In [57]:
train_image1 = []
train_image2 = []
for key in en_output.keys():
    train_image1.append(list(en_output[key][0]))
    train_image2.append(list(en_output[key][1]))
train_text = train.iloc[0:40,]['Clean_capt']

In [57]:
def create_vocab(text_df):
  vocab=[]
  for key in text_df.keys():
    vocab.extend(d.split() for d in text_df[key])
  words=[]
  for v_list in vocab:
    for word in v_list:
      words.append(word)
  return list(set(words))

vocab=create_vocab(text_dict)
vocab_size=len(vocab)

print('Vocabulary = %d' % (len(vocab)))

ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1

all_desc = list()
for text in text_dict:
    text=str(text)
    all_desc.append(text)
lines = all_desc
max_length = max(len(d.split()) for d in lines)

print('Description Length: %d' % max_length)


Vocabulary = 1353
Description Length: 125


In [49]:
def word_to_index(text_df):
  lines = []
  for key in text_df.keys():
    [lines.append(d) for d in text_df[key]]
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return(tokenizer.word_index)
  
word2index=word_to_index(text_dict)
idx2word = dict([(value, key) for key, value \
                 in word2index.items()]) 


In [ ]:
word2index

In [58]:
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8',\
                  newline='\n', errors='ignore')
    embedding = dict()
    for line in fin:
        tokens = line.rstrip().split(' ')
        embedding[tokens[0]] = np.array(tokens[1:],\
                                        dtype='float32')
    return embedding
embedding=load_vectors('/content/gdrive/MyDrive/Colab Notebooks/wiki.simple.vec')

def fastText(embedding):
  embedding_dim = 300
  embedding_matrix = np.zeros((vocab_size+1, \
                               embedding_dim))
  for word, i in word2index.items():
      embedding_vector = embedding.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
  return embedding_matrix,embedding_dim

fastText(embedding)
embedding_matrix,embedding_dim=fastText(embedding)

In [60]:
embedding_matrix

(1355, 300)

In [62]:
text_dict

0        [start no acute cardiopulmonary abnormality end]
1            [start no acute cardiopulmonary finding end]
2            [start no acute cardiopulmonary disease end]
3                                [start normal chest end]
4                                [start normal chest end]
                              ...                        
3209    [start borderline cardiomegaly no acute findin...
3210    [start heart size normal tortuous dilated aort...
3211         [start xxxx xxxx right pleural effusion end]
3212    [start decrease illdefined mixed lucent and op...
3213    [start moderate sized right pneumothorax there...
Name: Caption, Length: 3214, dtype: object

In [64]:
def create_train_data(text_values,max_length,vocab_size):
  X1,X2,y=list(),list(),list()
  for i in range(0,len(text_values)):
    for line in str(text_values[i]):
      numeric_seq = [word2index[word] for word in line.split()\
                     if word in word2index]
    for ii in range(1,len(numeric_seq)):
      in_seq,out_seq=numeric_seq[:ii],numeric_seq[ii]
      in_seq=pad_sequences([in_seq],maxlen=max_length,\
                           padding='post')[0]
      out_seq=to_categorical([out_seq],num_classes=vocab_size+1)[0]
      X2.append(in_seq)
      y.append(out_seq)
      X1.append(en_output['Number'][i])

  return (X1),(X2),(y)

X1,X2,y=create_train_data(text_dict,max_length,vocab_size)

In [71]:
batch_size=8
step_per_epoch=len(X1)//batch_size
epoch=30

def data_generator(X1,X2,y,batch_size,epoch,\
                   step_size,max_len,vocab_size):
  for j in range(0,epoch):
    for k in range(0,step_size):      
      for offset in range(0, len(X1), batch_size):
        batch_X1,batchX2,batchY=list(),list(),list()
        start_index=offset
        end_index=offset+batch_size
        for i in X1[start_index:end_index]:
          batch_X1.append(en_output[i][0][0])
        batchX2,batchY=X2[start_index:end_index],\
        y[start_index:end_index]
        yield [np.array(batch_X1),np.array(batchX2)],\
        np.array(batchY)
        
train_gen=data_generator(X1,X2,y,batch_size,epoch,\
                         step_per_epoch,max_length,vocab_size)

In [87]:
#image 
input_img = Input(shape=(4096,))
feature_img1 = Dropout(0.5)(input_img)
feature_img2 = Dense(256, activation='relu')(feature_img1)

#text
inputs_text = Input(shape=(max_length,))
feature_text1 = Embedding(vocab_size+1,embedding_dim,name='embedding_layer')(inputs_text)
feature_text2 = Dropout(0.5)(feature_text1)
feature_text3 = tf.keras.layers.GRU(256)(feature_text2)

#Concat
decoder1 = tf.keras.layers.Add()([feature_img2, feature_text3])
batch_norm= tf.keras.layers.BatchNormalization(momentum=0.99, epsilon=0.001)(decoder1)
decoder2 = Dense(256, activation='relu')(batch_norm)
outputs = Dense(vocab_size+1, activation='softmax')(decoder2)

model = Model(inputs=[input_img, inputs_text], outputs=outputs)
model.get_layer('embedding_layer').set_weights([embedding_matrix])
model.get_layer('embedding_layer').trainable = False

model.compile(optimizer='adam',loss='categorical_crossentropy',\
              metrics=['accuracy'])
model.summary()
filepath = 'ImageCaptioningModel.h5'
#checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', save_best_only=True,mode='min')

model.fit(train_gen, epochs=10, steps_per_epoch=100, 
          #callbacks=[checkpoint],
          verbose=1)
model.save_weights('/content/gdrive/MyDrive/Colab Notebooks/ImageCaptioningWeights.hdf5')


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_33 (InputLayer)          [(None, 125)]        0           []                               
                                                                                                  
 input_32 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 125, 300)     406500      ['input_33[0][0]']               
                                                                                                  
 dropout_29 (Dropout)           (None, 4096)         0           ['input_32[0][0]']               
                                                                                           

StopIteration: ignored

In [ ]:
def greedy_search(model,photo,vocab,max_len):
  output_text='start'
  while True:
    seq=[word2index[str(i)] for i in output_text.split()]
    seq=pad_sequences([seq],maxlen=max_len,padding='post')
    seq=np.array(seq[0])
    y_predict=model.predict([np.expand_dims(photo,axis=0),\
                             np.expand_dims(seq,axis=0)])
    output_text=output_text+' '+ idx2word[np.argmax(y_predict[0])]
    if (len(output_text.split())>max_len) or \
    ((idx2word[np.argmax(y_predict[0])]) == 'end'):
      break
  output_text=output_text.split()
  final_text=[' '.join(output_text[1:-1])]
  return final_text    

In [ ]:
train_image1[0]

In [26]:
c = train.iloc[0:20,]  ### using all test data instead
X_train_img = c['Number']  
y_train_rep = c['Clean_capt']


In [27]:
def load_image(id_, report):
    '''Loads the Image Features with their corresponding Ids'''
    img_feature = en_output[id_][0]
    return img_feature, report

In [33]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_img, y_train_rep))
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(load_image, [item1, item2],
                          [tf.float32, tf.string]),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
input1 = Input(shape=(2048), name='Image_1')
dense1 = Dense(256, kernel_initializer=tf.keras.initializers.glorot_uniform(seed = 56),
               name='dense_encoder')(input1)

input2 = Input(shape=(155), name='Text_Input')
emb_layer = Embedding(input_dim = vocab_size, output_dim = 300, input_length=155, mask_zero=True,
                      trainable=False, name="Embedding_layer") ##weights=[embedding_matrix], 
emb = emb_layer(input2)
LSTM2 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")
LSTM2_output = LSTM2(emb)
dropout1 = Dropout(0.5, name='dropout1')(LSTM2_output)

dec =  tf.keras.layers.Add()([dense1, dropout1])

fc1 = Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed = 63),
            name='fc1')
fc1_output = fc1(dec)
output_layer = Dense(vocab_size, activation='softmax', name='Output_layer')
output = output_layer(fc1_output)

encoder_decoder = Model(inputs = [input1, input2], outputs = output)

In [ ]:
!pip install autokeras

In [ ]:
input1 = Input(shape=(2048), name='Image_1')
dense1 = Dense(256, kernel_initializer=tf.keras.initializers.glorot_uniform(seed = 56),
               name='dense_encoder')(input1)

input2 = Input(shape=(155), name='Text_Input')
emb_layer = Embedding(input_dim = vocab_size, output_dim = 300, input_length=155, mask_zero=True,
                      trainable=False, name="Embedding_layer") ##weights=[embedding_matrix], 
emb = emb_layer(input2)

LSTM2 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")

LSTM2_output = LSTM2(emb)
dropout1 = Dropout(0.5, name='dropout1')(LSTM2_output)

dec =  tf.keras.layers.Add()([dense1, dropout1])

fc1 = Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed = 63),
            name='fc1')
fc1_output = fc1(dec)
output_layer = Dense(vocab_size, activation='softmax', name='Output_layer')
output = output_layer(fc1_output)
encoder_decoder = Model(inputs = [input1, input2], outputs = output)


In [28]:
import autokeras as ak

In [63]:
input_node1 = ak.ImageInput()
output_node1 = ak.Normalization()(input_node1)
output_node1 = ak.ImageAugmentation()(output_node1)
output_node1 = ak.ResNetBlock(version="v2")(output_node1)

input_node2 = ak.ImageInput()
output_node2 = ak.Normalization()(input_node2)
output_node2 = ak.ImageAugmentation()(output_node2)
#output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version="v2")(output_node2)

output_node1 = ak.Merge()([output_node1, output_node2])


input_node3 = ak.TextInput()
output_node3 = ak.TextToIntSequence()(input_node3)
output_node3 = ak.Embedding(pretraining='glove',embedding_dim=200)(output_node3)
# Use separable Conv layers in Keras.
output_node3 = ak.ConvBlock(separable=True)(output_node3)
# output_node3 = ak.ClassificationHead()(output_node3)


output_node = ak.Merge()([output_node1, output_node3])
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(inputs=[input_node1, input_node2, input_node3],outputs=output_node,overwrite=True,max_trials=2)


AttributeError: ignored

In [52]:
auto_model

In [53]:
auto_model.fit(
    [train_image1,train_image2,train_text],
    train_text,
    batch_size=32,
    epochs=3,verbose=1,
)

ValueError: ignored

In [33]:
x = np.array([[1, 2],[3,4]])
y = np.expand_dims(x, axis=0)
y

array([[[1, 2],
        [3, 4]]])